Problem set 5: Parsing
===========

This problem set contains two parts. 

- ** English Dependency parsing **: design features to learn a high-accuracy dependency parser for the English language
- ** Multilingual Dependency Parsing **: design features to learn a high-accuracy dependency parser for a language other than english of your choice

### Honor policy ###

- Your work must be your own. Do not discuss the details of the assignment with other people. 
- You may of course help each other with understanding the ideas discussed in lecture and the readings, and with basic questions about programming in Python. It is **not acceptable** to discuss how to implement a specific feature for dependency parsing, and it is unacceptable to share your code.
- There are implementations and source code for many machine learning algorithms on the internet. Please write the code for this assignment on your own, without using these external resources, except where noted (usually in the bakeoff only).

# Part 1: English Dependency parsing #

In this problem, you will work with an arc-factored non-projective dependency parser, which is trained by average perceptron. If you were not confident about your own implementation of averaged structure perceptron, please take a look at the code, in the directories shown below.

In [ ]:
import numpy as np
from os.path import join
import gtparsing
import gtparsing.dependency_parser as depp
import gtparsing.dependency_features as depf
import gtparsing.custom_features
import gtparsing.utilities
from score import accuracy
import matplotlib as plt
%pylab inline

In [ ]:
DIR = "data/deppars/"

In [ ]:
# build a dependency parser with a given feature set
dp = depp.DependencyParser(feature_function=depf.DependencyFeatures())
dp.read_data("english")

In [ ]:
# train your parser for ten iterations
# These are *unlabeled* accuracies.
dp.train_perceptron(10)

In file ```parsing/custom_features.py```, you are going to create a series of subclasses of ```DependencyFeatures```, which has features of your choice. We provide an example of one such class ```LexFeats```, which adds a feature that includes:

- The part-of-speech of the head
- The word of the modifier

** Note ** - Please take a careful look at the documentation provided for the class. It should help you design the other classes for the subsequent deliverables.

In [ ]:
importlib.reload(gtparsing.custom_features)
from gtparsing.custom_features import LexFeats
import importlib

In [ ]:
#let's run it
dp = depp.DependencyParser(feature_function=LexFeats())
dp.read_data("english")
dp.train_perceptron(1)

** Deliverable 1a ** (3 points): start by adding a feature that computes the distance between the head and the modifier, up to a maximum absolute value of 10. To do this, implement the code for the class ```LexDistFeats``` in ```gtparsing.custom_features```

**Sanity check**: you should now have 23039 features. Accuracy should improve substantially. 

In [ ]:
importlib.reload (gtparsing.custom_features)
from gtparsing.custom_features import LexDistFeats
import importlib

In [ ]:
dp = depp.DependencyParser(feature_function=LexDistFeats())
dp.read_data("english")
dp.train_perceptron(10)
dp.test(join (DIR, "english_dev.conll"), join (DIR, "deliverable1a.conll"))

** Deliverable 1b ** (3 points): now add a feature to LexDistFeats, which includes the POS of the modifier and the word of the head. To do this, implement the code for the class ```LexDistFeats2``` in ```gtparsing.custom_features```

**Sanity check**: The number of features should roughly double. (Do you see why?)

In [ ]:
importlib.reload (gtparsing.custom_features)
from gtparsing.custom_features import LexDistFeats2
import importlib

In [ ]:
dp = depp.DependencyParser(feature_function=LexDistFeats2())
dp.read_data("english")
dp.train_perceptron(10)
dp.test(join (DIR, "english_dev.conll"), join (DIR, "deliverable1b.conll"))

## Context features ## 

Add context features that consider the tags adjacent to the head and modifier. You may wish to consider various tag combinations, such as 

 - $\langle t[h], t[h-1], t[m]\rangle$: head, head-left, modifier
 - $\langle t[h], t[m], t[m+1]\rangle$: head, modifier, modifier-right
 - $\langle t[h], t[h-1], t[m], t[m+1]\rangle$: head, head-left, modifier, modifier-right
 - etc

Note that you can add more than one feature at a time within ```create_arc_features()```. Watch out for edge cases!

** Deliverable 1c ** (5 points):

Describe what context feature templates you have added. How do they impact the total number of features? How does 
it impact the development and training accuracy?

** Note **- You must atleast add head, head-left, modifier feature

To do this, implement the code for the class ```ContextFeats``` in ```gtparsing.custom_features```

** Sanity check **: I added a few basic context features, and this increased dev set accuracy above 82%.

In [ ]:
importlib.reload (gtparsing.custom_features)
from gtparsing.custom_features import ContextFeats
import importlib

In [ ]:
dp = depp.DependencyParser(feature_function=ContextFeats())
dp.read_data("english")
dp.train_perceptron(10)
dp.test(join (DIR, "english_dev.conll"), join (DIR, "deliverable1c.conll"))

Add your response to Deliverable 1c here

## Bakeoff! ##

Similar to the previous problem sets, we will have a kaggle competition for this problem set. Two days before the submission, we will release a test data set. Try to develop other features that will further improve performance. Please explain all the features that you have. We will provide the submission function through piazza.

After identifying your best feature set, run *test()* function from DependencyParser. Please name the file as **lastname-firstname.conll.pred**.

**Deliverable 1d** (4 points): Explain why the features you added would work and include your response file in your t-square submission.

**Sanity check / challenge**: The best test set performance in Fall 2014 was 86.5%. Can you beat it??

**NOTE**: As usual, you are not supposed to look at other code online while doing this problem set. However, you are welcome to search for *research papers* on dependency parsing to get ideas for features that might improve your performance.

In [ ]:
#Uncomment the line below and modify it so that the filename is lastname-firstname.conll.pred
DELIVERABLE1d = join (DIR, "lastname-firstname.conll.pred")

In [ ]:
# run this when you have trained on your best features. 
dp.test(join (DIR, "english_dev.conll"), DELIVERABLE1d)

# Part 2: Multilingual Dependency parsing #

In this part, you will do a series of tasks:

- **Delexicalized direct transfer learning**. This means training a parser without lexical features, and directly porting it to another languag, without retraining.
- Retrain the delexicalized parser in the target language.
- Train a lexicalized parser in the target language.

** Note ** - Running each part of this section can be slow, so make sure you start early if you want to finish in time.

First, you choose which target language you want to work with. Uncomment the line from the following cell depending on the language you choose.

In [ ]:
#TARGET_LANGUAGE = "german"
TARGET_LANGUAGE = "french"
#TARGET_LANGUAGE = "italian"
#TARGET_LANGUAGE = "spanish"
#TARGET_LANGUAGE = "portuguese"

To get us started, we warmup by comparing the probability distributions of part of speech tags for two different datasets.

- In part 1, you used the dataset identified by "english" language. 
- The same data is also available for "english_univtags" language. 
- The difference is in the size of the tagset, where the size is smaller in english_univtags. The tagset is chosen such that it has _universal_ applicability to multiple languages. Effectively, the tagging is much more coarse grained in english_univtags than in english.

** Deliverable 2a ** (3 points): Calculate the conditional probability distribution of modifier PoS tag given head PoS tag i.e $P(\text{modifier tag}=n \mid \text{head tag}=m), \forall n \in T$, where $T$ is the tagset.  In order to do this, implement the ```CPT``` function in ```gtparsing.utilities```, which should take as arguments a list of instances and the tag index representing the head of a dependency relation, and should return a dict of probabilities of the tags of the modifiers.

Use this function to calculate the conditional probability distributions where the head tag is verb for "english" and "english_univtags" datasets.

In [ ]:
importlib.reload(gtparsing.utilities)
from gtparsing.utilities import CPT
import importlib

In [ ]:
dp = depp.DependencyParser(feature_function=depf.DependencyFeatures())
dp.read_data("english")
eng_tag_dist = CPT(dp.reader.train_instances, dp.reader.pos_dict["VB"])
eng_reverse_lookup = {value:key for key,value in dp.reader.pos_dict.items()}

In [ ]:
dp = depp.DependencyParser(feature_function=depf.DependencyFeatures())
dp.read_data("english_univtags")
eng_univ_tag_dist = CPT(dp.reader.train_instances, dp.reader.pos_dict["VERB"])
eng_univ_reverse_lookup = {value:key for key,value in dp.reader.pos_dict.items()}

The following code plots the distributions.

In [ ]:
def plot_dist (values, labels):
    WIDTH = 0.35
    f = plt.figure(figsize = (10,5))
    ax = f.add_subplot(111)
    ax.bar(np.arange(len(values)),values)
    plt.xticks(np.arange(len(values)) + WIDTH, labels, rotation=45)

In [ ]:
plot_dist(list(eng_tag_dist.values()), [eng_reverse_lookup[tag_index] for tag_index in list(eng_tag_dist.keys())])
plot_dist(list(eng_univ_tag_dist.values()), [eng_univ_reverse_lookup[tag_index] for tag_index in list(eng_univ_tag_dist.keys())])

** Deliverable 2b ** (3 points) : Now calculate the [entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory) of each of the distribution. How are the two distributions different? 

In order to do this, you will implement ```entropy``` function ```gtparsing.utilities``` module 

In [ ]:
importlib.reload(gtparsing.utilities)
from gtparsing.utilities import entropy
import importlib

In [ ]:
print(entropy(eng_tag_dist))
print(entropy(eng_univ_tag_dist))

** Deliverable 2c ** (3 points) Implement ```DelexicalizedFeats``` in ```gtparsing/custom_features```. You should have the following features:

- part of speech tag of the head and modifier pair
- distance between the head and the modifier upto a maximum absolute value of 10

Train the dependency parser on ** english_univtags ** and transfer learning to the target language.

In [ ]:
importlib.reload(gtparsing.custom_features)
from gtparsing.custom_features import DelexicalizedFeats
import importlib

In [ ]:
# training on english
dp = depp.DependencyParser(feature_function=DelexicalizedFeats())
dp.read_data("english_univtags")
dp.train_perceptron(10)
dp.test(join (DIR, TARGET_LANGUAGE+"_dev.conll"), join (DIR, TARGET_LANGUAGE + ".deliverable2c.conll"))
print("Accuracy:", accuracy(join (DIR, TARGET_LANGUAGE+"_dev.conll"), join (DIR, TARGET_LANGUAGE + ".deliverable2c.conll")))

** Deliverable 2d **(3 points) Train the delexicalized parser in the target language. 

* Find the worst predicted label i.e the label for which the parser makes the most mistakes.
* Find the best predicted label ie. the label for which the parser makes the least mistakes,

In order to do this you may want to compare the predictions to the true labels by calculating the [confusion matrix](https://en.wikipedia.org/wiki/Confusion_matrix)

In [ ]:
dp = depp.DependencyParser(feature_function=DelexicalizedFeats())
dp.read_data(TARGET_LANGUAGE)
dp.train_perceptron(10)
dp.test(join (DIR, TARGET_LANGUAGE+"_dev.conll"), join (DIR, TARGET_LANGUAGE + ".deliverable2d.conll"))

**Deliverable 2e** (3 points) Now train the parser for 10 iterations using lexical features. Explain the following:

* Have the mistakes for the worst predicted label from 2d gone down? 
* Give top 3 worst predicted labels and top 3 best predicted labels. 

In [ ]:
# dp = depp.DependencyParser(feature_function=) # Make appropriate call here
dp.read_data(TARGET_LANGUAGE)
dp.train_perceptron(10)
dp.test(join (DIR, TARGET_LANGUAGE+"_dev.conll"), join (DIR, TARGET_LANGUAGE + ".deliverable2e.conll"))

** Deliverable 2f** (6 points) Train the parser for 10 iterations to improve the accuracy of the parser by at least 3%. Include at least one context feature and one morphological feature. The morphological features should access the words themselves; this is possible using the ```self.word_dict``` member variable of ```dependency_features```. Try to include more than one feature of each kind. Explain what features you added.

In [ ]:
# dp = depp.DependencyParser(feature_function=) # Make appropriate call here
dp.read_data(TARGET_LANGUAGE)
dp.train_perceptron(10)
dp.test(join (DIR, TARGET_LANGUAGE+"_dev.conll"), join (DIR, TARGET_LANGUAGE + ".deliverable2f.conll"))

# Part 3: 7650 Only #

- This is more research-oriented than in previous assignments. Be prepared to read and to think!
- [Hohensee and Bender](http://aclweb.org/anthology/N/N12/N12-1032.pdf) show that features measuring morphological  agreement can significantly improve dependency parsing.
- For your target language, read about morphology in your chosen target language.
- Design a set of features that capture morphological agreement between the head and the modifier. These features can condition on the part-of-speech, but they need to take the morphology into account too. For example, in English, you might want to capture subject-verb agreement. To do this you could design the following features:
    - < pos(h) = V, pos(m) = N, head verb is third-person singular, modifier noun is third-person singular>
    - < pos(h) = V, pos(m) = N, head verb is not third-person singular, modifier noun is plural>
    - etc
- In the example above, you could use simple morphology to distinguish whether the noun is plural and whether the verb agrees, for example by checking for 's' at the end of each word.
- If you can find existing resources (dictionaries or code) that capture morphology of interest for your targeted language, you may use them.

**Deliverable 3** (8 points) 

- Explain the morphological agreement that your features are trying to capture.
- Explain how you try to implement morphological agreement as a feature.
- Test whether your feature improves performance.